# Práctica 1: Reglas de asociación

Imports necesarios para la realización de la práctica

In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import mlxtend as mlx
import apyori as apy
import matplotlib as mpl
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [2]:
itemset = pd.read_csv("BlackFriday.csv").dropna().drop_duplicates()
itemset

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0
5,1000013,P00350442,M,46-50,1,C,3,1,2,3.0,15.0
6,1000013,P00155442,M,46-50,1,C,3,1,1,11.0,15.0
7,1000013,P0094542,M,46-50,1,C,3,1,2,4.0,9.0
8,1000015,P00161842,M,26-35,7,A,1,0,10,13.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...
233584,1006023,P00262242,M,26-35,0,C,2,1,1,11.0,16.0
233586,1006026,P00110742,M,36-45,6,C,1,1,1,2.0,8.0
233588,1006030,P00129842,M,26-35,17,C,1,1,6,8.0,15.0
233591,1006033,P00127642,M,51-55,13,B,1,1,1,2.0,15.0


Pregunta Nº1: Prueba al menos tres configuraciones de soporte y frecuencia para cada género, edad y tipo de producto. (1 punto)


In [3]:
transaccionesProduct = itemset[['User_ID', 'Product_ID', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']].groupby('User_ID').agg(lambda x: list(x)).reset_index()
transaccionesUser = itemset[['User_ID', 'Gender', 'Age']].groupby('User_ID').agg(lambda x: "".join(set(x))).reset_index()

transaccionesUser[['Product_ID', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']] = transaccionesProduct[['Product_ID', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']]
transacciones = transaccionesUser[['User_ID', 'Gender', 'Age', 'Product_ID', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']]

transacciones

,User_ID,Gender,Age,Product_ID,Product_Category_1,Product_Category_2,Product_Category_3
0,1000001,F,0-17,"[P00216342, P00070942, P0096442, P00144042, P0...","[3, 3, 4, 2, 3, 3]","[4.0, 5.0, 5.0, 3.0, 4.0, 4.0]","[5.0, 16.0, 12.0, 4.0, 5.0, 12.0]"
1,1000002,M,55+,"[P00182742, P00201442, P00303442, P00073842, P...","[1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1]","[2.0, 6.0, 8.0, 15.0, 2.0, 2.0, 14.0, 8.0, 2.0...","[14.0, 8.0, 14.0, 17.0, 15.0, 15.0, 16.0, 14.0..."
2,1000003,M,26-35,"[P00025442, P00253142, P00289942, P00062842, P...","[1, 1, 3, 1, 3, 1]","[2.0, 2.0, 4.0, 2.0, 4.0, 5.0]","[9.0, 5.0, 5.0, 13.0, 5.0, 18.0]"
3,1000005,M,26-35,"[P00081142, P00223542, P00270942, P00000642]","[6, 1, 1, 1]","[8.0, 8.0, 2.0, 6.0]","[14.0, 17.0, 15.0, 16.0]"
4,1000006,F,51-55,"[P00086042, P00128242, P00057542, P0094342, P0...","[8, 5, 3, 4, 3, 8]","[14.0, 12.0, 4.0, 5.0, 4.0, 14.0]","[17.0, 14.0, 5.0, 18.0, 12.0, 16.0]"
...,...,...,...,...,...,...,...
5640,1006036,F,26-35,"[P00002142, P00016842, P00127542, P00190142, P...","[1, 1, 1, 3, 2, 1, 1, 1, 2, 2, 6, 5, 11, 1, 1,...","[5.0, 2.0, 2.0, 4.0, 4.0, 8.0, 14.0, 15.0, 3.0...","[8.0, 15.0, 15.0, 5.0, 9.0, 17.0, 17.0, 16.0, ..."
5641,1006037,F,46-50,"[P00059442, P00144642, P00184942, P00112542, P...","[6, 1, 1, 1, 4, 3, 1, 5, 1]","[8.0, 15.0, 8.0, 11.0, 5.0, 4.0, 2.0, 13.0, 6.0]","[16.0, 16.0, 17.0, 15.0, 8.0, 5.0, 14.0, 16.0,..."
5642,1006038,F,55+,[P00112142],[1],[2.0],[14.0]
5643,1006039,F,46-50,"[P00088242, P00188742, P00282242, P00084742, P...","[6, 8, 5, 5, 4, 5, 4, 3, 10, 5, 8]","[10.0, 14.0, 8.0, 12.0, 5.0, 8.0, 8.0, 4.0, 14...","[13.0, 17.0, 17.0, 14.0, 12.0, 14.0, 9.0, 9.0,..."


In [4]:
tablaProductos = itemset[['Product_ID', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']].drop_duplicates()
tablaProductos

,Product_ID,Product_Category_1,Product_Category_2,Product_Category_3
4,P00053842,4,5.0,12.0
5,P00350442,2,3.0,15.0
6,P00155442,1,11.0,15.0
7,P0094542,2,4.0,9.0
8,P00161842,10,13.0,16.0
...,...,...,...,...
88454,P00290742,8,14.0,18.0
101475,P0099942,5,12.0,14.0
112297,P00243042,5,10.0,16.0
176536,P00296842,1,5.0,6.0


In [6]:
te = TransactionEncoder()
te_ary = te.fit(transacciones['Product_ID'].to_list()).transform(transacciones['Product_ID'].to_list())
df = pd.DataFrame(te_ary, columns=te.columns_)
df


frequent_itemsets = apriori(df, min_support=0.003, use_colnames=True)
# K=1 -> 137 items
# K=2 -> 
frequent_itemsets


,support,itemsets
0,0.085740,(P00000142)
1,0.030115,(P00000242)
2,0.037733,(P00000642)
3,0.036670,(P00001042)
4,0.044464,(P00001142)
...,...,...
16508,0.003543,"(P00154042, P00237542, P00242742)"
16509,0.003189,"(P00192042, P00321742, P00221442)"
16510,0.003012,"(P00242742, P00221442, P00277642)"
16511,0.003012,"(P00237542, P00242742, P00277442)"


Pregunta Nº2: ¿Qué diferencias hay entre usar soporte y frecuencia? Respalda la respuesta con datos (1 punto)

Pregunta Nº3: ¿Qué tipo reglas desaparecen según la configuración y categorías (género, edad y tipo de producto) consideradas? ¿Por qué? (1 punto)

Pregunta Nº4: Para una de las configuraciones, interpreta algunas de las reglas que te hayan resultado interesantes obtenidas usando la clase association_rules y un par de configuraciones cambiando la variable metric y min_threshold. Justifica los resultados. (1 punto)


Pregunta Nº5: Dadas las mejores configuraciones. ¿Existen reglas o patrones que se repitan? ¿Podemos generalizar de alguna manera como se comportan los clientes? Expón las conclusiones respaldadas con datos (1 punto)